In [1]:
import os
import pandas as pd
from tqdm import tqdm  # For progress tracking
from cftsdata import abr

# Define paths
source_dir = os.path.abspath(
    "../../../../data/practicum-data/ABRpresto data/ABRpresto data")
destination_dir = os.path.abspath(
    "../../../data/practicum-data/ABRpresto data/processed_data")
output_file = os.path.join(destination_dir, "updated_processed_waveforms.csv")

# Ensure destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Initialize list to store processed data
all_data = []

# Get list of folders in the source directory
folders = [f for f in os.listdir(
    source_dir) if os.path.isdir(os.path.join(source_dir, f))]


In [2]:
def extract_metadata(folder_name):
    """Extract mouse_id, time_point, and ear from folder name."""
    parts = folder_name.split('_')
    mouse_id = parts[0].replace('Mouse', '')
    time_point = parts[1].replace('timepoint', '')
    ear = parts[2]  # Assumes ear is always in the third part
    return mouse_id, time_point, ear


In [3]:
def process_folders_in_batches(folders, batch_num, total_batches=10):
    """Processes a specific batch of folders based on batch number."""
    batch_size = len(folders) // total_batches
    start_idx = batch_num * batch_size
    end_idx = (batch_num + 1) * batch_size if batch_num < total_batches - 1 else len(folders)

    print(f"Processing batch {batch_num + 1}/{total_batches}: Folders {start_idx} to {end_idx}")

    for folder in tqdm(folders[start_idx:end_idx], desc=f"Batch {batch_num + 1} Progress"):
        folder_path = os.path.join(source_dir, folder)
        mouse_id, time_point, ear = extract_metadata(folder)

        # Ensure only the first part of 'ear' is used (assuming it’s separated by non-alphanumeric characters)
        ear = ear.split()[0] if isinstance(ear, str) else ear  # Keeps only the first part

        try:
            # Print folder name for debugging
            print(f"Processing folder: {folder}")

            # Load ABR data
            try:
                fh = abr.load(folder_path)
            except Exception as e:
                print(f"Skipping {folder} due to error in abr.load: {e}")
                continue

            try:
                epochs = fh.get_epochs_filtered()
                if epochs.empty:
                    raise ValueError("Epochs data is empty")
                epochs_mean = epochs.groupby(['frequency', 'level']).mean()
            except Exception as e:
                print(f"Skipping {folder} due to error in get_epochs_filtered: {e}")
                continue

            # Process each frequency-level pair
            for (freq, level), group in epochs_mean.iterrows():
                wave_data = group.values  # Replace with actual waveform column if needed

                # Convert wave array into separate columns with integer keys
                wave_dict = {i + 1: val for i, val in enumerate(wave_data)}

                # Append to list
                all_data.append({
                    'mouse_id': mouse_id,
                    'time_point': time_point,
                    'ear': ear,
                    'frequency': freq,
                    'level': level,
                    **wave_dict
                })
        except Exception as e:
            print(f"Error processing {folder}: {e}")


In [4]:
process_folders_in_batches(folders, batch_num=0)

Processing batch 1/10: Folders 0 to 110


Batch 1 Progress:   0%|          | 0/110 [00:00<?, ?it/s]

Processing folder: Mouse155_timepoint1_left abr_io


Batch 1 Progress:   1%|          | 1/110 [00:01<03:18,  1.82s/it]

Processing folder: Mouse336_timepoint0_right abr_io


...

Processing folder: Mouse137_timepoint0_left abr_io


Batch 1 Progress:  73%|███████▎  | 80/110 [01:54<00:41,  1.38s/it]

Processing folder: Mouse97_timepoint0_left abr_io


Batch 1 Progress:  73%|███████▎  | 80/110 [01:55<00:43,  1.45s/it]


KeyboardInterrupt: 

In [5]:
all_data_df = pd.DataFrame(all_data)
all_data_df.head()

""


In [ ]:
# # Create DataFrame and save as CSV
# if all_data:
#     df = pd.DataFrame(all_data)
#     df.to_csv(output_file, index=False)
#     print(f"Processing complete. CSV saved to {output_file}")
# else:
#     print("No data was processed. CSV was not created.")